# Experiments with ObsLocTAP for the Rubin scheduler viewer for survey operations

## 1. Introduction
This notebook demonstrates how to look at the ObsLocTAP results that are being published by the [Rubin schedule viewer](https://usdf-rsp.slac.stanford.edu/obsloctap/static/viewer.html) for the Rubin Scheduler Viewer.
This notebook follows a lot of the similar ground as the "official" [102 Rubin Schedule Viewer notebook](https://github.com/lsst/tutorial-notebooks/blob/6c28f8f532f4852ed34d12e9662b3f7f086d4453/Commissioning/102_rubin_schedule_viewer.ipynb)

### 1.1 Import needed packages

In [ ]:
from datetime import datetime, timedelta

import numpy as np
from astropy.time import Time
from astropy.table import QTable
from astropy import units as u
from astropy.coordinates import SkyCoord
import pandas as pd
import requests
from sregion import SRegion
import skyproj
import healpy as hp
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

%matplotlib widget


## 2. Rubin Schedule Viewer

The Rubin Schedule Viewer runs at the US Data Facility at SLAC.

Define the ObsLocTAP URL of the service.

In [ ]:
obsloctap_url = 'https://usdf-rsp.slac.stanford.edu/obsloctap'

### 2.1 Programmatic access

The service is available programatically through the ObsLocTAP service, allowing users to obtain the publicly distributed observing schedule.

In [ ]:
response = requests.get(obsloctap_url)
assert response.status_code == 200, f'request failed with status {response.status_code}'
print(f'Rubin Schedule Viewer API at {response.url} is alive.')

In [ ]:
headers = response.json()
assert headers is not None
meta_data = headers['metadata']
print(meta_data.keys())

## 2.2 Retrieve the next night's schedule
Define observation date, convert to string in the right format and the number of hours to fetch (12 hours in this case(  and fetch the VOTable of scheduled observations which are in [ObsLocTAP](https://www.ivoa.net/documents/ObsLocTAP/index.html) format. We then convert the three time-related columns to AstroPy `Time` mixin columns.

In [ ]:
obs_date = datetime(2026, 1, 23)
date_str = obs_date.strftime('%Y-%m-%d')
schedule_hours = 12
params = {'time': schedule_hours, 'start': date_str}
print(f'Searching for {schedule_hours} hours of observations on {date_str}')

In [ ]:
schedule_url = schedule_url = obsloctap_url + '/schedule'
response = requests.get(schedule_url, params=params)
assert response.status_code == 200, f'request failed with status {response.status_code}'
print(response.url)

Retrieve the forecasted schedule as an Astropy `QTable` (could also read it as a  Pandas DataFrame using `next_visits = pd.DataFrame(response.json())`)  and print the number of forecasted visits.

In [ ]:
next_visits = QTable.read(schedule_url, format='pandas.json')
print(f"There are {len(next_visits)} visits scheduled in the next {params['time']} hours.")

Display the top 10 of the planned visits in the next hours

In [ ]:
next_visits[0:10]

In [ ]:
for column in ['t_planning', 't_min', 't_max']:
    t = Time(next_visits[column], format='mjd')
    next_visits[column] = t

In [ ]:
next_visits

### Footprints and regions
The footprint information is in the `s_region` column and is in the unformalized but widely used subset of [STC-S](https://www.ivoa.net/documents/Notes/STC-S/) as detailed in Section 6 of the [TAP 1.0](https://www.ivoa.net/documents/TAP/20100327/REC-TAP-1.0.html) specification. We can use the `sregion` package (need >=1.5 for the `BOX` support TL added) to turn these into `SRegion` objects which can then be converted into `matplotlib.patch` patches or `shapely` polygons

In [ ]:
row = next_visits[0]
if row['s_region'] != '':
    sr = SRegion(row['s_region'])
else:
    sr = SRegion(f"CIRCLE {row['s_ra']} {row['s_dec']} {row['s_fov']}", ncircle=256)
print(sr.centroid)
print(sr.sky_area(u.deg**2)[0])

### Plotting
We use `skyproj` to make a Mollweide projection plot and label it with the Milky Way band and the ecliptic

In [ ]:
plt.close()
# Plotting options; default is black lines, 1.5 linewidth, 10 point labels
linewidth = 1.5
color = 'black'
label_size = 10

fig, ax = plt.subplots(1, 1, layout='constrained', dpi=150)
sp = skyproj.MollweideSkyproj(ax=ax)
sp.ax.set_xlabel('Right Ascension', fontsize=label_size)
sp.ax.set_ylabel('Declination', fontsize=label_size)
# Draw Milky Way and +/- 10 degrees of the equator

sp.draw_milky_way(label='Milky Way', linewidth=linewidth, color=color)
mw_line = sp.ax.lines[-3]
# Draw ecliptic and label
elon = np.linspace(0, 360, 500)
elat = np.zeros_like(elon)
ec = SkyCoord(lon=elon * u.degree, lat=elat * u.degree, distance=1 * u.au, frame='heliocentricmeanecliptic')
radec = ec.icrs
lon = radec.ra.degree
lat = radec.dec.degree
sp.ax.plot(lon, lat, linewidth=1.0, color='green', linestyle='--', label='Ecliptic')
ecl_line = sp.ax.lines[-1]
legend_handles = [mw_line, ecl_line]
sp.ax.legend(
    bbox_to_anchor=(1, 1), bbox_transform=fig.transFigure, handles=legend_handles, loc='upper right', fontsize='x-small'
)
ax.grid()

### Load in the Rubin SV survey area and plot that first

In [ ]:
hpix_map = hp.read_map('sv_skymap_healpix.fits')
_ = sp.draw_hpxmap(hpix_map, zoom=False)
# QuadMeshes can't be added to legends so we need to make Patches manually
cmap = plt.get_cmap()
sv_ecl_patch = mpatches.Patch(color=cmap.colors[-1], label='SV survey (Ecl. area)')
sv_lvk_patch = mpatches.Patch(color=cmap.colors[0], label='SV survey (LVK area)')
legend_handles.append(sv_lvk_patch)
legend_handles.append(sv_ecl_patch)

In [ ]:
# Load Deep Drilling Fields and special Roman field locations
from fomo_nb_utils import special_locations

ddfs = special_locations()
for ddf_name, coords in ddfs.items():
    circle_fill = True
    label = ''
    color = 'yellow'
    if 'Roman' in ddf_name:
        circle_fill = False
        label = 'Roman'
        color = 'orange'
    elif 'COSMOS' in ddf_name:
        label = 'DDFs'
    print(f'{ddf_name}: RA= {coords[0]:.3f}, Dec = {coords[1]:+.3f}')
    # Draw circle with a little larger radius of 1.9 degrees (vs 1.75) to account for dithers
    ddf_poly = sp.ax.circle(coords[0], coords[1], 1.9, color=color, linewidth=linewidth, fill=circle_fill, label=label)
    if label:
        legend_handles.append(ddf_poly[0])

### Plot all the rows (Rubin scheduled fields) as circles

In [ ]:
for ra, dec, radius in zip(next_visits['s_ra'], next_visits['s_dec'], next_visits['s_fov']):
    sp.ax.circle(ra, dec, radius, fill=True, color='red', alpha=0.1, linewidth=0.75, linestyle='--')
night_patches = sp.ax.lines[-1]
night_patches.set_label(date_str)
legend_handles.append(night_patches)

In [ ]:
# Update legend and add title
sp.ax.legend(
    bbox_to_anchor=(1, 1), bbox_transform=fig.transFigure, handles=legend_handles, loc='upper right', fontsize='x-small'
)
sp.ax.set_title(f'Rubin scheduled fields for {date_str}')